In [2]:
# Copyright 2023 DeepMind Technologies Limited
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [3]:
# Skip this cell if running from local environment
!pip install -q --no-warn-conflicts git+https://github.com/google-deepmind/nuclease_design.git
# After installing, restart the kernel
# Then run cells as normal

# Correlation between liquid culture and EF from HTS


In [6]:
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd
import seaborn as sns

from nuclease_design import utils

In [7]:
# Load enrichment factors
g1df = utils.load_data('g1')
g2df = utils.load_data('g2')
g3df = utils.load_data('g3')
g4df = utils.load_data('g4')

In [8]:
tierdf = pd.read_csv('../data/liquid_culture.csv', index_col=0)
tierdf = tierdf.dropna()

In [9]:
def parse_mutations_from_genotype(genotype: str):
    if not genotype:
        return None
    if 'frameshift' in genotype:  # No frameshifts in high-throughput screening data
        print(genotype)
        return None
    if 'KanR' in genotype: # backbone mutation, not needed
        print(genotype)
        return None
    if 'N129N' in genotype: # What is this genotype?
        print(genotype)
        return None
    if genotype == 'wild type':
        return ()
    mutations = genotype.split(',')
    return tuple((m[0], int(m[1:-1]), m[-1]) for m in mutations)

assert parse_mutations_from_genotype("A73R") == (("A", 73, "R"), )
assert parse_mutations_from_genotype("A63P,A73R,D74H,I84Y") == (("A", 63, "P"), ("A", 73, "R"), ("D", 74, "H"), ("I", 84, "Y"))


In [10]:
tierdf['mutations'] = tierdf.genotype.apply(parse_mutations_from_genotype)

A73R,N129N
A73R,N129N
A73R,N129N
A73R,N129N
A73R,N129N
A73R,N129N
A73R,N129N
A73R,N129N
A73R,N129N
A63S,D64S,A73R,KanR
W5frameshift,A63S,D64S,A73R
A24frameshift,D38E,S45K,S55A,A63S,A73R,D74F


In [11]:
# correlations:
# within a dataset, within a genotype
dataset_to_df = {}
for dataset in tierdf.dataset.unique():
    datasetdf = tierdf[tierdf["dataset"] == dataset].dropna()
    # get mean and std
    dataset_to_df[dataset] = datasetdf.groupby('mutations').agg(mean_activity=('activity', 'mean'), 
                                                                std_activity=('activity', 'std')).reset_index()

In [12]:
def mutations_to_name(mutations):
    if len(mutations) == 0:
        return "WT"
    return ','.join(f'{a}{b}{c}' for a, b, c in mutations)

In [13]:
landscapedf = utils.load_landscape()

In [14]:
dataset_ef_pairs = [('G1 Tier2', g1df), 
                    ('ML2 Tier1', g2df), 
                    ('ML2 Tier2', g2df), 
                    ('ML3 Tier1', g3df), 
                    ('ML4 Tier1', g4df)]

# hue_order = ['non-functional', 'activity_greater_than_0', 'activity_greater_than_WT', 'activity_greater_than_A73R']
# palette = sns.color_palette("Greys", 1) + sns.color_palette(n_colors=3)

hue_order = ['activity_greater_than_A73R', 'activity_greater_than_WT', 'activity_greater_than_0', 'non-functional']
palette = sns.color_palette("cubehelix", n_colors=4)


activity_to_color = {k: v for k, v in zip(hue_order, palette)}
n_cols = 4
n_rows = 6 # G1, G1, G2, G2 , G3, G4


fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*3), dpi=500)


skipped = set()
row_i = 0
for dataset_name, efdf in dataset_ef_pairs:
    mergedf = dataset_to_df[dataset_name].merge(efdf, on='mutations', how='left')
    mergedf = mergedf.merge(landscapedf[['mutations', 'activity_level']], on='mutations', how='left')
    ef_cols = [col for col in efdf.columns if col.startswith('ef_')]
    n_cols = len(ef_cols)
    # create vertical space
    for col_j, col in enumerate(ef_cols):
        if col_j > 3:
            row_i = 1
            col_j = col_j % 4
        for _, row in mergedf.iterrows():
            if np.isnan(row[col]):
                if row['mutations'] not in skipped:
                    skipped.add(row['mutations'])
                    print(f"Skipping {row['mutations']}, no EF")
                continue
            if not row['std_activity'] > 0:
                axes[row_i, col_j].plot(row[col], row['mean_activity'], 'x', color=activity_to_color[row['activity_level']])
            else:
                axes[row_i, col_j].errorbar(row[col], row['mean_activity'], yerr=row['std_activity'], fmt='.', color=activity_to_color[row['activity_level']])
            # add text label for mutations with short names
            if len(row['mutations']) > 1:
                continue
            else:
                offset = 1
                axes[row_i, col_j].text(row[col], row['mean_activity'], f"{mutations_to_name(row['mutations'])}", rotation=45)
        axes[row_i, col_j].set_xlim(left=0)
        axes[row_i, col_j].set_ylim(bottom=0)
        axes[row_i, col_j].set_ylabel(f"liquid culture activity")
        axes[row_i, col_j].set_xlabel("enrichment factor")
        axes[row_i, col_j].set_title(f"{dataset_name}\nvs\n{col}")
        axes[row_i, col_j].set_xlim(left=0)
    # delete any unused axes
    
    for ax in axes[row_i, col_j+1:]:
        fig.delaxes(ax)
    row_i += 1
        

### begin legend
color_patches = [mpatches.Patch(color=color, label=f'{description}') for description, color in activity_to_color.items()]

# Create custom legend entries for symbols
# dot_line = mlines.Line2D([], [], color='black', marker='.', linestyle='None', markersize=10, label='Multiple replicates')
x_line = mlines.Line2D([], [], color='black', marker='x', linestyle='None', markersize=10, label='Single liquid culture replicate')

# Combine the color patches and symbol lines into a single legend
legend_handles = color_patches + [x_line]

fig.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(1, 1), title='Landscape activity level')
### end legend
fig.subplots_adjust(wspace=0.5, hspace=1.2)
# tight layout
plt.tight_layout()
plt.show()

Skipping (('D', 74, 'R'), ('G', 125, 'K')), no EF
Skipping (('A', 73, 'R'), ('D', 74, 'S')), no EF
Skipping (('A', 34, 'E'), ('D', 38, 'E'), ('S', 45, 'T'), ('S', 55, 'A'), ('A', 63, 'S'), ('D', 64, 'E'), ('A', 73, 'R'), ('D', 74, 'S'), ('Q', 78, 'A'), ('K', 82, 'R'), ('E', 101, 'V')), no EF
Skipping (('D', 38, 'E'), ('S', 45, 'Q'), ('A', 60, 'K'), ('A', 63, 'S'), ('A', 73, 'R'), ('D', 74, 'F'), ('K', 87, 'R'), ('E', 101, 'V')), no EF
Skipping (('D', 38, 'E'), ('S', 45, 'Q'), ('S', 55, 'R'), ('K', 59, 'N'), ('A', 63, 'P'), ('D', 64, 'E'), ('A', 73, 'R'), ('D', 74, 'H'), ('K', 82, 'R'), ('I', 84, 'Y'), ('K', 87, 'R')), no EF
Skipping (('R', 35, 'L'), ('V', 39, 'I'), ('D', 64, 'S'), ('A', 73, 'R'), ('D', 74, 'W'), ('Q', 78, 'A')), no EF
